In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
import torch
import torch.nn

Matplotlib is building the font cache; this may take a moment.


In [2]:
emb_ent = torch.load('../LiterallyWikidata/files_needed/pretrained_kge/pretrained_complex_entemb.pt')
print(emb_ent.shape)
list_ent_ids =[]
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as f:
    for line in f:
        list_ent_ids.append(line.strip())
print(len(list_ent_ids))

attri_data = pd.read_csv('../LiterallyWikidata/files_needed/nogeo_df48_var')

#attri_data['e'][0] --> find list idx --> emb.ent[idx]
ent2idx = {e:i for i,e in enumerate(list_ent_ids)}
attri_data['list_idx']=attri_data['e'].map(ent2idx)


weight = emb_ent
embedding = torch.nn.Embedding.from_pretrained(weight)
input = torch.LongTensor(attri_data['list_idx'].to_numpy())


torch.Size([47998, 128])
47998


In [5]:
attri_data[:5]

,e,a,v,name_e,name_a,ent_type,std_v,list_idx
0,Q1000056,P1082,11032.0,Sušice,population,Q7841907,-0.030708,5
1,Q1000056,P2044,472.0,Sušice,elevation above sea level,Q7841907,-0.031042,5
2,Q1000056,P2046,45630000.0,Sušice,area,Q7841907,1.411756,5
3,Q1000138,P1082,1375.0,Cantenac,population,Q484170,-0.031014,8
4,Q1000138,P2044,1.0,Cantenac,elevation above sea level,Q484170,-0.031057,8


In [6]:
weight = emb_ent
embedding = torch.nn.Embedding.from_pretrained(weight)
input = torch.LongTensor(attri_data['list_idx'].to_numpy())

In [22]:
with open('../LiterallyWikidata/files_needed/saved_all2idx.pkl','r')as fr:
    dict_all2idx = pickle.load(fr)

FileNotFoundError: [Errno 2] No such file or directory: 'LiterallyWikidata/files_needed/saved_all2idx.pkl'

In [9]:
emb_e = embedding(input)

In [18]:
len(attri_data['a'].unique())

81

In [20]:
embedding_a = torch.nn.Embedding(len(attri_data['a'].unique()), 128, padding_idx=0)
embedding_a

Embedding(81, 128, padding_idx=0)

In [24]:
x = attri_data.loc[:,['e','a']]
y= attri_data.loc[:,'std_v']

0        -0.030708
1        -0.031042
2         1.411756
3        -0.031014
4        -0.031057
            ...   
153088   -0.031000
153089   -0.030994
153090   -0.030994
153091   -0.030995
153092   -0.030999
Name: std_v, Length: 153093, dtype: float64

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.33, random_state=802)

In [27]:
nn=MLPRegressor(hidden_layer_sizes=(6,6,6),  #預設值1層,數目100
         activation='logistic',  #{'identity'(回歸)無，'logistic'為sigmoid，'tanh'，'relu'}，預設='relu'
         solver='sgd',   #{'lbfgs'牛頓，'sgd'隨機梯度，'adam'隨機梯度優化}，預設='adam'
         batch_size='auto',  #'auto'為min(200～樣本數)
         learning_rate='constant',  #constant固定'，'invscaling隨著時間遞減'，'adaptive誤差減少時不改變'}，預設 ='constant' 
         learning_rate_init=0.01, #優化器為sgd、adam使用
         power_t=0.5,     # 優化器為sgd，則可以設定強化學習率   
         max_iter=1000,    #訓練次數
         shuffle=True,    #隨機設定權重
         random_state=1,   #隨機的基礎值
         momentum=0.9)

In [28]:
nn.fit(X_train, y_train)
y_predtrain=nn.predict(X_train)

ValueError: could not convert string to float: 'Q103244'